In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import yaml
from sklearn.metrics import f1_score, accuracy_score

/tmp/ipykernel_88001/3454234130.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2024-02-23 17:28:06.173827: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-23 17:28:06.173935: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-23 17:28:06.314370: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable 

In [3]:
config_path = './config.yaml'
with open(config_path, 'r') as file:
    config = yaml.safe_load(file)


def load_data(data_type):
    data_path = f"{config['directories']['preprocessed_data']}/train_embeddings_{data_type}.pkl"
    label_path = f"{config['directories']['preprocessed_data']}/train_labels_{data_type}.pkl"
    data_df = pd.read_pickle(data_path)
    label_df = pd.read_pickle(label_path)
    return data_df, label_df

datasets = {category: load_data(category) for category in ['BiologicalProcesses', 'CellularComponent', 'MolecularFunction']}

In [25]:
# Assuming parent_path is the parent directory of model_root_path
models_path = config["directories"]["models"]

model_data_map = {
    'BiologicalProcesses': 'best_BP_model',
    'CellularComponent': 'best_CC_model',
    'MolecularFunction': 'best_MF_model'
}

In [35]:
model_folders = [d for d in os.listdir(models_path) if os.path.isdir(os.path.join(models_path, d))]
model_folders

['CNNMod1', 'FFNNMod1', 'FFNNMod2', 'MLPMod']

In [37]:
best_models = {}

for category in datasets:
    data_df, label_df = datasets[category]
    best_f1_score = 0
    best_model_path = None
    best_accuracy = 0
    
    print(f"Evaluating models for {category}:")
    for model_folder in model_folders:
        print(f'Evaluating {model_folder}...')
        current_model_path = f'{models_path}/{model_folder}/{model_data_map[category]}'
        
        model = tf.keras.models.load_model(current_model_path)
        
        scores = model.evaluate(data_df, label_df, verbose=0)
        accuracy = scores[1]
        precision = scores[3]
        recall = scores[4]
        F1_score = 2*precision*recall / (precision + recall)
      
        print(f"Model: {model_folder}, Accuracy: {accuracy}, F1 Score: {F1_score}")
        
        if F1_score > best_f1_score:
            best_f1_score = F1_score
            best_accuracy = accuracy
            best_model_path = current_model_path
            
    best_models[category] = (best_model_path, best_accuracy, best_f1_score)
    
for category, (path, accuracy, f1) in best_models.items():
    print(f"\nBest model for {category}: {path} with Accuracy: {accuracy}, F1 Score (Sample Avg): {f1}")
        

Evaluating models for BiologicalProcesses:
Evaluating CNNMod1...
Model: CNNMod1, Accuracy: 0.9752015471458435, F1 Score (Sample Avg): 0.4387889154153564
Evaluating FFNNMod1...
Model: FFNNMod1, Accuracy: 0.9770002961158752, F1 Score (Sample Avg): 0.46438130478447454
Evaluating FFNNMod2...
Model: FFNNMod2, Accuracy: 0.976239800453186, F1 Score (Sample Avg): 0.4425751018222649
Evaluating MLPMod...
Model: MLPMod, Accuracy: 0.9750373959541321, F1 Score (Sample Avg): 0.3984541818322404
Evaluating models for CellularComponent:
Evaluating CNNMod1...
Model: CNNMod1, Accuracy: 0.9806859493255615, F1 Score (Sample Avg): 0.47545037052783257
Evaluating FFNNMod1...
Model: FFNNMod1, Accuracy: 0.9811740517616272, F1 Score (Sample Avg): 0.4972965738268135
Evaluating FFNNMod2...
Model: FFNNMod2, Accuracy: 0.9806178212165833, F1 Score (Sample Avg): 0.4962278912594993
Evaluating MLPMod...
Model: MLPMod, Accuracy: 0.9797471165657043, F1 Score (Sample Avg): 0.44674299981993265
Evaluating models for Molecula

OSError: SavedModel file does not exist at: ./models/CNNMod1/best_MF_model/{saved_model.pbtxt|saved_model.pb}